### Markov Decision Process

In [1]:
import numpy as np
np.random.seed(0)

#### Markov Process
1. Analytical solution
2. Dynamic programming
3. Monte-Carlo method
4. Temporal Difference

In [7]:
P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]
P = np.array(P)

rewards = [-1, -2, -2, 10, 1, 0] # s0: guard index; s6: stop state
gamma = 0.5

def total_return(start_index, chain, gamma):
    G = 0
    for i in reversed(range(start_index, len(chain))):
        # 秦九韶算法
        G = gamma * G + rewards[chain[i] - 1] # rewards
    return G

chain = [1, 2, 3, 6]
start_index = 0
G = total_return(start_index, chain, gamma)
print("The return of this chain is {:4f}".format(G))

The return of this chain is -2.500000


#### Bellman equation

In [12]:
def analyticl_solution(P, rewards, states_num, gamma):
    rewards = np.array(rewards).reshape(-1, 1)
    value_matrix = np.dot(np.linalg.inv(np.eye(states_num) - gamma * P), rewards)
    return value_matrix

value_matrix = analyticl_solution(P, rewards, P.shape[0], gamma)
print("V(s) of each state = \n", value_matrix)

V(s) of each state = 
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


#### Markov decision process

In [16]:
# state set
S = ["s1", "s2", "s3", "s4", "s5"]  
# action set
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  
# State transfer function: P(s'|s, a)，in state s + action a → state s'
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
}
# Reward Function
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
}
gamma = 0.5  # 
MDP = (S, A, P, R, gamma) 

# Policy 1
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
}
# Policy 2
Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
}

def join(str1, str2):
    return str1 + '-' + str2

In [17]:
gamma = 0.5
# P from MDP to MRP
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]
P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp)
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0]

V = analyticl_solution(P_from_mdp_to_mrp, R_from_mdp_to_mrp, 5, gamma)
print("V(s) of each state in MDP = \n", V)

V(s) of each state in MDP = 
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


#### Monte-Carlo methods

In [19]:
def sample(MDP, Pi, timestep_max, number):
    """
    MDP: markov decision process
    Pi: the policy used
    timestep_max: 当状态为终止状态或时间步太长，采样终止
    number: the number of sampling
    """
    S, A, P, R, gamma =  MDP
    episodes = []
    for _ in range(number):
        episode = []
        timestep = 0
        s = S[np.random.randint(4)]
        while (s != 's5') and timestep <= timestep_max:
            timestep += 1
            rand, temp = np.random.rand(), 0
            # choose action in state s
            for a_opt in A:
                temp += Pi.get(join(s, a_opt), 0) # if s + a_opt donnot exsist, return 0
                if temp > rand:
                    a = a_opt
                    r = R.get(join(s, a), 0)
                    break
            # transfer to next state depend on P(s'|s, a)
            rand, temp = np.random.rand(), 0
            for s_opt in S:
                temp += P.get(join(join(s, a), s_opt), 0)
                if temp > rand:
                    s_next = s_opt
                    break
            episode.append((s, a, r, s_next))
            s = s_next
        episodes.append(episode)
    return episodes

#### test ####
episodes = sample(MDP, Pi_1, 20, 5)
print("The first episode: {}\n".format(episodes[0]))
print("The third episode: {}\n".format(episodes[2]))
print("The fifth episode: {}\n".format(episodes[4]))

The first episode: [('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '前往s5', 10, 's5')]

The third episode: [('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '概率前往', 1, 's2'), ('s2', '前往s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '前往s5', 10, 's5')]

The fifth episode: [('s4', '前往s5', 10, 's5')]



In [21]:
def Monte_Carlo(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        for i in range(len(episode) - 1, -1, -1):
            s, a, r, s_next = episode[i]
            G = r + gamma * G
            N[s] += 1
            V[s] += (G - V[s]) / N[s]

timestep_max = 20
episodes = sample(MDP, Pi_1, timestep_max, 1000)
V = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
N = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
Monte_Carlo(episodes, V, N, gamma)            
print("The V(s) caculated by MC = {}".format(V))    

The V(s) caculated by MC = {'s1': -1.212969652449074, 's2': -1.670482336856089, 's3': 0.524151625860306, 's4': 6.084119574026449, 's5': 0}


#### Occupancy

In [24]:
import math
def occupancy(episodes, s_target, a_target, timestep_max, gamma):
    rho = 0
    total_times = np.zeros(timestep_max)
    occur_times = np.zeros(timestep_max)
    for episode in episodes:
        for i in range(len(episode)):
            s, a, r, s_next = episode[i]
            total_times[i] += 1
            if (s == s_target) and (a == a_target):
                occur_times[i] += 1
    for i in reversed(range(timestep_max)):
        if (total_times[i]):
            ### 公式和实际存在差别
            rho += math.pow(gamma, i) * occur_times[i] / total_times[i]
    return (1 - gamma) * rho

gamma = 0.5
timestep_max = 1000

episodes_1 = sample(MDP, Pi_1, timestep_max, 1000)
episodes_2 = sample(MDP, Pi_2, timestep_max, 1000)
rho_1 = occupancy(episodes_1, "s4", "概率前往", timestep_max, gamma)
rho_2 = occupancy(episodes_2, "s4", "概率前往", timestep_max, gamma)
print(rho_1, rho_2)

0.11267436262461648 0.2312320427566883
